In [1]:
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Activation, Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from onset_detection.read_data import read_data

Using Theano backend.
DEBUG: nvcc STDOUT nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
mod.cu
   Creating library D:/Temp/theano_compiledir/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmp06x85pvh/m91973e5c136ea49268a916ff971b7377.lib and object D:/Temp/theano_compiledir/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmp06x85pvh/m91973e5c136ea49268a916ff971b7377.exp

Using gpu device 0: GeForce GTX 980M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5005)


In [2]:
active_datasets = {'ds1', 'ds2', 'ds3', 'ds4'}
X, y = read_data(active_datasets)
print(X.shape)
print(y.shape)
print(sum(y))

D:\Users\Michel\Documents\FH\module\8_IP6\git\onset_detection\read_data.py:150: UserWarning: No truth found for AR_Lick11_FN.wav, skipping file.
  warn('No truth found for ' + wav_file + ', skipping file.')
D:\Users\Michel\Documents\FH\module\8_IP6\git\onset_detection\read_data.py:150: UserWarning: No truth found for AR_Lick11_KN.wav, skipping file.
  warn('No truth found for ' + wav_file + ', skipping file.')
D:\Users\Michel\Documents\FH\module\8_IP6\git\onset_detection\read_data.py:150: UserWarning: No truth found for AR_Lick11_MN.wav, skipping file.
  warn('No truth found for ' + wav_file + ', skipping file.')
D:\Users\Michel\Documents\FH\module\8_IP6\git\onset_detection\read_data.py:152: UserWarning: Skipping non-wav file data\IDMT-SMT-GUITAR_V2\dataset2\audio\desktop.ini
  warn('Skipping non-wav file ' + path_to_wav)
D:\Users\Michel\Documents\FH\module\8_IP6\git\onset_detection\read_data.py:150: UserWarning: No truth found for FS_Lick11_FN.wav, skipping file.
  warn('No truth foun

(1171926, 111)
(1171926,)
51536


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)
print(X_train.mean())
print(X_train.std())
print(X_test.mean())
print(X_test.std())

(937540, 111)
(937540,)
(234386, 111)
(234386,)
1.60520331603e-19
1.0
-9.37002541738e-05
1.00155776089


In [4]:
input_dim = X_train.shape[1]
X_train = X_train.reshape(X_train.shape[0], input_dim, 1)
X_test = X_test.reshape(X_test.shape[0], input_dim, 1)
input_shape = (input_dim, 1)
print(X_train.shape)
print(X_test.shape)
print(input_shape)

(937540, 111, 1)
(234386, 111, 1)
(111, 1)


In [7]:
def create_model(input_shape=(111, 1), dropout=False):
    nb_filter = 32
    filter_length = 8
    border_mode = 'same'
    pool_length = 2
    
    model = Sequential()
    model.add(Convolution1D(nb_filter, filter_length,
                            border_mode=border_mode,
                            input_shape=input_shape,))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_length=pool_length))
    if dropout:
        model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    if dropout:
        model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',)
    
    return model

clf = KerasClassifier(
    build_fn=create_model,
    batch_size=1024, nb_epoch=500,
    validation_data=(X_test, y_test),
    callbacks=[EarlyStopping(monitor='loss', patience=2)],
    input_shape=input_shape,
)
clf.fit(X_train, y_train)
y_train_predicted = clf.predict(X_train)
y_test_predicted = clf.predict(X_test)

Train on 937540 samples, validate on 234386 samples
Epoch 1/500
937540/937540 [==============================] - 16s - loss: 0.1559 - val_loss: 0.1344
Epoch 2/500
937540/937540 [==============================] - 16s - loss: 0.1276 - val_loss: 0.1240
Epoch 3/500
937540/937540 [==============================] - 16s - loss: 0.1205 - val_loss: 0.1198
Epoch 4/500
937540/937540 [==============================] - 16s - loss: 0.1168 - val_loss: 0.1166
Epoch 5/500
937540/937540 [==============================] - 16s - loss: 0.1148 - val_loss: 0.1156
Epoch 6/500
937540/937540 [==============================] - 16s - loss: 0.1127 - val_loss: 0.1157
Epoch 7/500
937540/937540 [==============================] - 17s - loss: 0.1113 - val_loss: 0.1141
Epoch 8/500
937540/937540 [==============================] - 16s - loss: 0.1099 - val_loss: 0.1132
Epoch 9/500
937540/937540 [==============================] - 17s - loss: 0.1085 - val_loss: 0.1131
Epoch 10/500
937540/937540 [=============================

In [8]:
print(classification_report(y_train, y_train_predicted))
print(classification_report(y_test, y_test_predicted))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99    896320
          1       0.91      0.82      0.86     41220

avg / total       0.99      0.99      0.99    937540

             precision    recall  f1-score   support

          0       0.97      0.98      0.98    224070
          1       0.50      0.44      0.47     10316

avg / total       0.95      0.96      0.95    234386



In [9]:
clf = KerasClassifier(
    build_fn=create_model,
    batch_size=1024, nb_epoch=500,
    validation_data=(X_test, y_test),
    callbacks=[EarlyStopping(monitor='loss', patience=2)],
    input_shape=input_shape, dropout=True,
)
clf.fit(X_train, y_train)
y_train_predicted = clf.predict(X_train)
y_test_predicted = clf.predict(X_test)

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/Temp/theano_compiledir/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmpfpg9l6rs/md953f6e2da014848347663cc100f1fd6.lib and object D:/Temp/theano_compiledir/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmpfpg9l6rs/md953f6e2da014848347663cc100f1fd6.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/Temp/theano_compiledir/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmp8ty16h_5/mc8064d39203a98af4ed4d35a8d09bf0b.lib and object D:/Temp/theano_compiledir/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmp8ty16h_5/mc8064d39203a98af4ed4d35a8d09bf0b.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/Temp/theano_compiledir/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmpjt5su9yh/m497f9daba605b74

Train on 937540 samples, validate on 234386 samples
Epoch 1/500
  4096/937540 [..............................] - ETA: 20s - loss: 0.5765

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/Temp/theano_compiledir/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmppnfmri5n/mda0425657fec5f52bc6b79603f6788cd.lib and object D:/Temp/theano_compiledir/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64/tmppnfmri5n/mda0425657fec5f52bc6b79603f6788cd.exp



937540/937540 [==============================] - 17s - loss: 0.1666 - val_loss: 0.1409
Epoch 2/500
937540/937540 [==============================] - 17s - loss: 0.1430 - val_loss: 0.1312
Epoch 3/500
937540/937540 [==============================] - 17s - loss: 0.1356 - val_loss: 0.1266
Epoch 4/500
937540/937540 [==============================] - 17s - loss: 0.1315 - val_loss: 0.1242
Epoch 5/500
937540/937540 [==============================] - 17s - loss: 0.1288 - val_loss: 0.1220
Epoch 6/500
937540/937540 [==============================] - 17s - loss: 0.1265 - val_loss: 0.1214
Epoch 7/500
937540/937540 [==============================] - 17s - loss: 0.1249 - val_loss: 0.1193
Epoch 8/500
937540/937540 [==============================] - 17s - loss: 0.1234 - val_loss: 0.1188
Epoch 9/500
937540/937540 [==============================] - 17s - loss: 0.1222 - val_loss: 0.1172
Epoch 10/500
937540/937540 [==============================] - 17s - loss: 0.1212 - val_loss: 0.1168
Epoch 11/500
937540/9

In [10]:
print(classification_report(y_train, y_train_predicted))
print(classification_report(y_test, y_test_predicted))

             precision    recall  f1-score   support

          0       0.97      1.00      0.98    896320
          1       0.86      0.29      0.44     41220

avg / total       0.96      0.97      0.96    937540

             precision    recall  f1-score   support

          0       0.97      1.00      0.98    224070
          1       0.82      0.27      0.41     10316

avg / total       0.96      0.97      0.96    234386

